In [514]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, GRU, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
from keras.utils import to_categorical
from keras import backend as K
from keras.models import Model
from keras.models import load_model
import random
from scipy.spatial.distance import cosine

In [2]:
from platform import python_version
print(python_version())

3.6.6


### Exp of classifing movie genres based on their events.


In [319]:
movies = pd.read_csv('../../data/metadata_events_table.tsv', sep = '\t')

In [320]:
print(len(movies))

13017


In [349]:
smr_path = ('../../data/raw/MovieSummaries/indiv_summaries/')
smr_list = [item for item in os.listdir(smr_path) if 'txt' in item]

In [351]:
d = {}
for item in smr_list:
    d[item.split('.')[0]] = open(os.path.join(smr_path, item), 'r').read()

In [357]:
movies.Wikipedia_id = movies.Wikipedia_id.astype(str)
movies['Summary'] = movies['Wikipedia_id'].map(d)

In [456]:
movies.head(2)

,Wikipedia_id,Freebase_id,Name,Release_date,Revenue,Runtime,Languages,Countries,Genres,Events,vecs,Summary
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","['send', 'hold', 'find', 'learn', 'murder', 'f...","[1.0, -0.8731093406677246, -0.0, -0.9997652173...","Set in the second half of the 22nd century, th..."
1,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}","['draw', 'learn', 'meet', 'fall', 'enter', 'fi...","[0.0, -0.0, -0.7400037050247192, -0.0, -0.0, 0...","Eva, an upper class housewife, becomes frustra..."


### Encode as one-hot vectors

In [162]:
t = Tokenizer()

In [163]:
events = []
for e in movies.Events.tolist():
    events.append([item.split('-')[0] for item in eval(e)])

In [164]:
# Size of data
print(len(events))

13017


In [165]:
t.fit_on_texts(events)

In [166]:
# t.word_index

In [167]:
vocab_size = max(t.word_index.values())+1

In [168]:
vocab_size

3608

In [172]:
msk = np.random.rand(len(movies)) < 0.9
train = []
for e in movies[msk]['Events'].tolist():
    train.append([item.split('-')[0] for item in eval(e)])
test = []
for e in movies[~msk]['Events'].tolist():
    test.append([item.split('-')[0] for item in eval(e)])

In [173]:
train = t.texts_to_sequences(train)
train = pad_sequences(train, maxlen=70)
# x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train = train[:,:-1]
y_train = train[:,-1]
y_train = to_categorical(y_train, num_classes=vocab_size)

In [174]:
test = t.texts_to_sequences(test)
test = pad_sequences(x_train, maxlen=70)
# x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = test[:,:-1]
y_test = test[:,-1]
y_test = to_categorical(y_test, num_classes=vocab_size)

In [186]:
events_trans = t.texts_to_sequences(events)
events_trans = pad_sequences(events_trans, maxlen=70)
# events_trans = np.reshape(events_trans, (events_trans.shape[0], events_trans.shape[1], 1))
x_all = events_trans[:,:-1]

In [188]:
x_all.shape

(13017, 69)

### Simple LSTM 

In [ ]:
# embedding_vector_length = 20
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=69))
model.add(Dropout(0.2))
model.add( LSTM(128))
model.add(Dense(vocab_size, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=500, batch_size=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 69, 20)            72160     
_________________________________________________________________
dropout_2 (Dropout)          (None, 69, 20)            0         
_________________________________________________________________
lstm_58 (LSTM)               (None, 128)               76288     
_________________________________________________________________
dense_31 (Dense)             (None, 3608)              465432    
Total params: 613,880
Trainable params: 613,880
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
11714/11714 [==============================] - 15s 1ms/step - loss: 6.5524 - acc: 0.0166
Epoch 2/500
11714/11714 [==============================] - 11s 962us/step - loss: 5.9748 - acc: 0.0196
Epoch 3/500
11714/11714 [==============================] - 

11714/11714 [==============================] - 11s 920us/step - loss: 4.2650 - acc: 0.1440
Epoch 72/500
11714/11714 [==============================] - 11s 966us/step - loss: 4.2468 - acc: 0.1485
Epoch 73/500
11714/11714 [==============================] - 12s 984us/step - loss: 4.2237 - acc: 0.1512
Epoch 74/500
11714/11714 [==============================] - 11s 979us/step - loss: 4.2039 - acc: 0.1543
Epoch 75/500
11714/11714 [==============================] - 11s 941us/step - loss: 4.1817 - acc: 0.1582
Epoch 76/500
11714/11714 [==============================] - 11s 955us/step - loss: 4.1576 - acc: 0.1615
Epoch 77/500
11714/11714 [==============================] - 11s 934us/step - loss: 4.1344 - acc: 0.1652
Epoch 78/500
11714/11714 [==============================] - 11s 921us/step - loss: 4.1094 - acc: 0.1683
Epoch 79/500
11714/11714 [==============================] - 11s 918us/step - loss: 4.0881 - acc: 0.1709
Epoch 80/500
11714/11714 [==============================] - 11s 923us/step - 

11714/11714 [==============================] - 11s 950us/step - loss: 3.0356 - acc: 0.3379
Epoch 150/500
11714/11714 [==============================] - 11s 954us/step - loss: 3.0244 - acc: 0.3382
Epoch 151/500
11714/11714 [==============================] - 11s 969us/step - loss: 3.0152 - acc: 0.3394
Epoch 152/500
11714/11714 [==============================] - 11s 969us/step - loss: 3.0072 - acc: 0.3442
Epoch 153/500
11714/11714 [==============================] - 11s 936us/step - loss: 3.0180 - acc: 0.3400
Epoch 154/500
11714/11714 [==============================] - 11s 929us/step - loss: 3.0234 - acc: 0.3380
Epoch 155/500
11714/11714 [==============================] - 11s 919us/step - loss: 2.9812 - acc: 0.3461
Epoch 156/500
11714/11714 [==============================] - 11s 920us/step - loss: 2.9568 - acc: 0.3488
Epoch 157/500
11714/11714 [==============================] - 11s 922us/step - loss: 2.9539 - acc: 0.3527
Epoch 158/500
11714/11714 [==============================] - 11s 921u

11714/11714 [==============================] - 11s 938us/step - loss: 2.3699 - acc: 0.4596
Epoch 228/500
11714/11714 [==============================] - 11s 922us/step - loss: 2.3576 - acc: 0.4657
Epoch 229/500
11714/11714 [==============================] - 11s 922us/step - loss: 2.3548 - acc: 0.4654
Epoch 230/500
11714/11714 [==============================] - 11s 921us/step - loss: 2.3487 - acc: 0.4683
Epoch 231/500
11714/11714 [==============================] - 11s 935us/step - loss: 2.3455 - acc: 0.4643
Epoch 232/500
11714/11714 [==============================] - 11s 967us/step - loss: 2.3433 - acc: 0.4670
Epoch 233/500
11714/11714 [==============================] - 11s 973us/step - loss: 2.3310 - acc: 0.4667
Epoch 234/500
11714/11714 [==============================] - 11s 924us/step - loss: 2.3298 - acc: 0.4677
Epoch 235/500
11714/11714 [==============================] - 11s 925us/step - loss: 2.3230 - acc: 0.4705
Epoch 236/500
11714/11714 [==============================] - 11s 923u

11714/11714 [==============================] - 13s 1ms/step - loss: 1.9665 - acc: 0.5369
Epoch 306/500
11714/11714 [==============================] - 12s 1ms/step - loss: 1.9663 - acc: 0.5377
Epoch 307/500
11714/11714 [==============================] - 12s 1ms/step - loss: 1.9408 - acc: 0.5492
Epoch 308/500
11714/11714 [==============================] - 17s 1ms/step - loss: 1.9380 - acc: 0.5513
Epoch 309/500
11714/11714 [==============================] - 13s 1ms/step - loss: 1.9196 - acc: 0.5540
Epoch 310/500
11714/11714 [==============================] - 12s 1ms/step - loss: 1.9324 - acc: 0.5522
Epoch 311/500
11714/11714 [==============================] - 14s 1ms/step - loss: 1.9240 - acc: 0.5488
Epoch 312/500
11714/11714 [==============================] - 14s 1ms/step - loss: 1.9226 - acc: 0.5557
Epoch 313/500
11714/11714 [==============================] - 13s 1ms/step - loss: 1.9364 - acc: 0.5470
Epoch 314/500
11714/11714 [==============================] - 12s 1ms/step - loss: 1.920

11714/11714 [==============================] - 15s 1ms/step - loss: 1.6521 - acc: 0.6047
Epoch 385/500
11714/11714 [==============================] - 14s 1ms/step - loss: 1.6549 - acc: 0.6000
Epoch 386/500
11714/11714 [==============================] - 14s 1ms/step - loss: 1.6456 - acc: 0.6006
Epoch 387/500
11714/11714 [==============================] - 15s 1ms/step - loss: 1.6587 - acc: 0.6039
Epoch 388/500
11714/11714 [==============================] - 14s 1ms/step - loss: 1.6498 - acc: 0.6058
Epoch 389/500
11714/11714 [==============================] - 15s 1ms/step - loss: 1.6488 - acc: 0.6072
Epoch 390/500
11714/11714 [==============================] - 14s 1ms/step - loss: 1.6494 - acc: 0.6047
Epoch 391/500
11714/11714 [==============================] - 15s 1ms/step - loss: 1.6333 - acc: 0.6082
Epoch 392/500
11714/11714 [==============================] - 15s 1ms/step - loss: 1.6202 - acc: 0.6137
Epoch 393/500
11714/11714 [==============================] - 15s 1ms/step - loss: 1.643

In [516]:
# Final evaluation of the model
scores = model.evaluate(x_test,
                        y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

11714/11714 [==============================] - 4s 363us/step
Accuracy: 1.57%


In [485]:
model.save('lstm.h5')

In [461]:
model = load_model('lstm.h5')

### Obtain embeddings

In [184]:
# #Alternative way, same result
# intermediate_layer_model = Model(inputs=model.input,
#                                  outputs=model.get_layer('lstm_53').output)


In [486]:
#Get indimediate layer output using tensorflow backend
get_lstm_output = K.function([model.layers[0].input], [model.layers[1].output])
output = get_lstm_output([x_all])

In [487]:
vecs = output[0]

In [501]:
rd = random.choice(range(len(vecs)))
rd

5782

In [502]:
# rd = 2457

In [503]:
# movies[movies.Name == 'Star Wars Episode VI: Return of the Jedi']

In [504]:
vecs_d = np.delete(vecs, rd, axis=0)

In [505]:
def find_closest(vc, dest):
    dist = 1
    min_idx = 0
    for i in range(len(vc)):
        dist_u = cosine(dest, vc[i])
        if dist_u < dist:
            dist = dist_u
            min_idx = i
    return dist, min_idx

In [506]:
rd_c = find_closest(vecs_d, vecs[rd])

In [507]:
rd_c

(0.054032862186431885, 7470)

In [508]:
movies.iloc[rd]

Wikipedia_id                                             24821605
Freebase_id                                            /m/08082jc
Name                                     The High and the Flighty
Release_date                                           1956-02-18
Revenue                                                       NaN
Runtime                                                       NaN
Languages                                                      {}
Countries                                                      {}
Genres          {"/m/02hmvc": "Short Film", "/m/0hqxf": "Famil...
Events          ['see', 'take', 'stuff', 'leave', 'look', 'see...
vecs            [0.40684789419174194, -0.0, -0.0, -0.0, -1.0, ...
Summary         Daffy Duck is a traveling salesman for the Ace...
Name: 5782, dtype: object

In [509]:
movies.iloc[rd].Summary

'Daffy Duck is a traveling salesman for the Ace Novelty Company of Walla Walla, Washington, when he witnesses Foghorn Leghorn and The Barnyard Dog in one of their familiar alternating scraps. Foghorn is seen awakening the dog by lifting him up by the tail and repeatedly slapping his rear end with a board which causes the dog to chase him. The dog is in pursuit but reaches a painted white line with a sign that reads "Rope Limit" which causes the dog to be jerked to a stop by the rope around his neck. Foghorn then takes a rubber ball and stuffs it in the dog\'s mouth, then punctures the ball with a needle causing the dog to fly away. As Foghorn leaves he walks past a wooden tower with a sign the reads "Don\'t Look Up". Foghorn naturally looks and sees the dog perched on the tower holding a watermelon which the dog releases, causing it to break over Foghorn\'s head. The dog\'s prank prompts Foghorn to contemplate "massive retaliation" against his nemesis. Daffy enters with his traveling s

In [510]:
movies.iloc[rd].Events

"['see', 'take', 'stuff', 'leave', 'look', 'see', 'prompt', 'contemplate', 'get', 'want', 'get', 'set', 'see', 'join', 'say']"

In [511]:
movies.iloc[rd_c[1]]

Wikipedia_id                                             18497134
Freebase_id                                            /m/04f0rdd
Name                                                        Dev D
Release_date                                           2008-12-19
Revenue                                                       NaN
Runtime                                                       144
Languages       {"/m/03k50": "Hindi Language", "/m/064_8sq": "...
Countries                                   {"/m/03rk0": "India"}
Genres          {"/m/03q4nz": "World cinema", "/m/04t36": "Mus...
Events          ['meet', 'turn', 'disown', 'work', 'adopt', 'r...
vecs            [0.39704257249832153, -0.08544650673866272, -0...
Summary         Dev  is the son of a rich businessman. He and ...
Name: 7470, dtype: object

In [512]:
movies.iloc[rd_c[1]].Events

"['meet', 'turn', 'disown', 'work', 'adopt', 'reserve', 'find', 'forget', 'call', 'visit', 'stay', 'show', 'spurn', 'go', 'decide', 'seek']"

In [513]:
movies.iloc[rd_c[1]].Summary

"Dev  is the son of a rich businessman. He and Paro  are childhood sweethearts. But Dev, being an insecure narcissist, instead of acknowledging her affection and care, nudges Paro over frivolous things. Dev is sent to London for higher studies when his father senses how spoilt his son is. While separated by distance, Paro and Dev's youthful love only blossoms more. Dev arrives in Chandigarh to meet Paro. Their endeavor to make love makes for some dark comic moments. The seeds of suspicion are sown here, which the couple will never be able to weed out for a lifetime. When Dev hears rumours about Paro, he immediately believes them and ditches her. What makes them fall apart is mutual suspicion and an essentially male vision of how a woman should conduct herself sexually. Paro turns her back on him when she hears him insult her and agrees to marry whoever her parents choose. On her wedding day, he realizes that the rumors were false. But his ego doesn't let him accept his mistake, and he 